# Chapter 13: Summarizing and Aggregating

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.13.0

## Group By Context

In [ ]:
import polars as pl

top2000 = pl.read_excel(
    "data/top2000-2023.xlsx", read_options={"skip_rows": 1}
).set_sorted("positie")

In [ ]:
(
    top2000.group_by("jaar")
    .agg(  
        songs=pl.concat_str(
            pl.col("artiest"), pl.lit(" - "), pl.col("titel")
        ),  
    )
    .sort("jaar", descending=True)
)

### The Descriptives

In [ ]:
(
    top2000.group_by("jaar", maintain_order=True)  
    .head(3)  
    .sort("jaar", descending=True)
    .head(9)  
)

In [ ]:
(
    top2000.group_by("jaar", maintain_order=True)
    .tail(3)
    .sort("jaar", descending=True)
    .head(9)
)

In [ ]:
(top2000.group_by("artiest").len().sort("len", descending=True).head(10))

In [ ]:
df = pl.read_csv("data/sales.csv")
df.columns

In [ ]:
(
    df.select("Product_Category", "Sub_Category", "Unit_Price")  
    .group_by("Product_Category", "Sub_Category")  
    .max()
    .sort("Unit_Price", descending=True)  
    .head(10)
)

In [ ]:
(
    df.select("Country", "Profit")
    .group_by("Country")
    .sum()
    .sort("Profit", descending=True)
)

In [ ]:
(
    df.select("Sub_Category", "Product")
    .group_by("Sub_Category")
    .n_unique()
    .sort("Product", descending=True)
    .head(10)
)

In [ ]:
(
    df.select("Age_Group", "Order_Quantity")
    .group_by("Age_Group")
    .mean()
    .sort("Order_Quantity", descending=True)
)

In [ ]:
(
    df.select("Age_Group", "Revenue")
    .group_by("Age_Group")
    .quantile(0.9)
    .sort("Revenue", descending=True)
)

### The Advanced

### Aggregate Values to a List

In [ ]:
(
    df.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit"),
        pl.col("Revenue"),
    )
)

### Rename Aggregated Columns

In [ ]:
(
    df.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit").alias("All Profits Per Transactions"),
        pl.col("Revenue").name.prefix("All "),
        Cost=pl.col("Revenue") - pl.col("Profit"),
    )
)

### Apply Multiple Aggregations At Once

In [ ]:
(
    df.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit").sum().name.prefix("Total "),
        pl.col("Profit").mean().alias("Average Profit per Transaction"),
        pl.col("Revenue").sum().name.prefix("Total "),
        pl.col("Revenue").mean().alias("Average Revenue per Transaction"),
    )
)

In [ ]:
(
    df.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.all().sum().name.prefix("Total "),
        pl.all().mean().name.prefix("Average "),
    )
)

In [ ]:
(
    df.select("Country", "Profit")
    .group_by("Country")
    .agg(
        (pl.col("Profit") > 1000).alias("Profit > 1000"),
        (pl.col("Profit") > 1000)
        .sum()
        .alias("Number of Transactions with Profit > 1000"),
    )
)

In [ ]:
def custom_agg(col: str) -> pl.Expr:
    col1 = (col > 1000).alias("Profit > 1000")
    col2 = (col > 1000).sum().alias("Number of Transactions with Profit > 1000")
    return col1, col2


(
    df.select("Country", "Profit")
    .group_by("Country")
    .agg(custom_agg(pl.col("Profit")))
)

## Row-wise Aggregations

In [ ]:
fold_example = pl.DataFrame({"col1": [2], "col2": [3], "col3": [4]})

fold_example.with_columns(
    sum=pl.fold(
        acc=pl.lit(0),  
        function=lambda acc, x: acc + x,  
        exprs=pl.col("*"),  
    )
)

In [ ]:
products = pl.DataFrame(
    {
        "product_A": [10, 20, 30],
        "product_B": [20, 30, 40],
        "product_C": [30, 40, 50],
    }
)

weights = {"product_A": 0.5, "product_B": 1.5, "product_C": 2.0}  

weighted_exprs = [  
    (pl.col(product) * weight).alias(product)
    for product, weight in weights.items()
]

products_with_weighted_sum = products.with_columns(
    weighted_sum=pl.fold(  
        acc=pl.lit(0),  
        function=lambda acc, x: acc + x,  
        exprs=weighted_exprs,  
    )
)

products_with_weighted_sum

## Window Functions in Selection Context

In [ ]:
(
    top2000.select(
        "jaar",
        "artiest",
        "titel",
        "positie",
        year_rank=pl.col("positie").rank().over("jaar"),
    ).sample(10, seed=42)
)

## Dynamic Grouping

## Rolling Aggregations

In [ ]:
from datetime import date

dates = pl.date_range(  
    start=date(2024, 4, 1),
    end=date(2024, 4, 26),
    interval="2d",
    eager=True,  
)
dates = dates.filter(dates.dt.weekday() < 6)  
dates_repeated = pl.concat([dates, dates]).sort()  

small_sales_df = (
    pl.DataFrame(
        {
            "date": dates_repeated,
            "store": ["Store A", "Store B"] * dates.len(),
            "sales": [
                200, 150, 220, 160, 250, 180, 270, 190, 280, 210,
                210, 170, 220, 180, 240, 190, 250, 200, 260, 210,
            ],
        }
    )
    .set_sorted("date")  
    .set_sorted("store")
)

In [ ]:
result = small_sales_df.rolling(  
    index_column="date",
    period="7d",
    group_by="store",
).agg(  
    sum_of_last_7_days_sales=pl.sum("sales")
)

final_df = small_sales_df.join(result, on=["date", "store"])  

final_df

## Upsampling

In [ ]:
upsampled_small_sales_df = small_sales_df.upsample(
    time_column="date", every="1d", group_by="store", maintain_order=True
)

upsampled_small_sales_df

In [ ]:
upsampled_small_sales_df.select(
    "date", pl.col("store").forward_fill(), pl.col("sales").interpolate()
)

## Takeaways